In [1]:
!pip install beautifulsoup4 langchain-community tiktoken langchainhub langchain langgraph tavily-python langchain-openai python-dotenv black isort pytest
!pip install langchain-chroma langchain_google_genai langchain-tavily==0.1.5 

#

In [2]:
import os
from dotenv import load_dotenv
import tqdm as notebook_tqdm
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader

C:\Users\gayat\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
load_dotenv()

True

In [4]:
from typing import List, TypedDict

class GraphState(TypedDict):
    """State object for workflow containing query, documents, and control flags."""
    question: str           # User's original query
    generation: str         # LLM-generated response  
    web_search: bool       # Control flag for web search requirement
    documents: List[str]   # Retrieved document context

#### Create the models

In [5]:
llm_model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",                     
    temperature=0,                                
)

embed_model = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

#### Create web search tool using Tavily

In [7]:
from langchain.schema import Document
from langchain_tavily import TavilySearch
from typing import Any, Dict

web_search_tool = TavilySearch(max_results=3)

def web_search(state: GraphState) -> Dict[str, Any]:
    print("---WEB SEARCH---")
    question = state["question"]
    
    documents = state.get("documents", [])  # Get existing documents or empty list
    
    tavily_results = web_search_tool.invoke({"query": question})["results"]
    joined_tavily_result = "\n".join(
        [tavily_result["content"] for tavily_result in tavily_results]
    )
    web_results = Document(page_content=joined_tavily_result)
    
    # Add web results to existing documents (or create new list if documents was empty)
    if documents:
        documents.append(web_results)
    else:
        documents = [web_results]
    
    return {"documents": documents, "question": question}

#### Create vectore store using chroma

In [8]:
def create_vectorstore():
    """Create or load vector store for document retrieval."""
    chroma_path = "./chroma_langchain_db"
    
    if os.path.exists(chroma_path):
        print("Loading existing vector store...")
        vectorstore = Chroma(
            persist_directory=chroma_path,
            embedding_function=embed_model,
            collection_name="rag-chroma",
        )
        return vectorstore.as_retriever()
    
    print("Creating new vector store...")
    urls = [
        "https://www.webmd.com/covid/coronavirus",
        "https://www.webmd.com/diabetes/type-1-diabetes",
        "https://www.webmd.com/diabetes/type-2-diabetes",
        "https://www.webmd.com/a-to-z-guides/understanding-anemia-basics/",
    ]

    docs = [WebBaseLoader(url).load() for url in urls]
    docs_list = [item for sublist in docs for item in sublist]

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=250, 
        chunk_overlap=0
    )
    doc_splits = text_splitter.split_documents(docs_list)

    vectorstore = Chroma.from_documents(
        documents=doc_splits,
        collection_name="rag-chroma",
        embedding=embed_model,
        persist_directory=chroma_path,
    )
    
    print("Vector store created!")
    return vectorstore.as_retriever()

retriever = create_vectorstore()

Loading existing vector store...


In [10]:
retriever.invoke("Symptoms of covid")

[Document(id='791289f3-91a9-4942-9104-8b65ee34f795', metadata={'description': 'COVID-19 is a new type of coronavirus that causes mild to severe cases. Here’s a quick guide on how to spot symptoms, risk factors, prevent spread of the disease, and find out what to do if you think you have it.', 'title': 'Coronavirus & COVID-19 Overview: Symptoms, Risks, Prevention, Treatment & More', 'language': 'en', 'source': 'https://www.webmd.com/covid/coronavirus'}, page_content='COVID-19 SYMPTOM CHECKER\n\t\t\t\t\t\t\t\t\t\t\t\n\n\n\n                                                        <bulletlist><bulletitem><btitle>Find Out if You Have Symptoms of Coronavirus (COVID-19)</btitle><description><p><a href="https://www.webmd.com/coronavirus/coronavirus-assessment/default.htm">See what to do about your symptoms and whether to call a doctor</a></p></description></bulletitem></bulletlist>'),
 Document(id='ddc98749-f2c0-4a04-816f-8f8e35c8f50c', metadata={'source': 'https://www.webmd.com/covid/coronavir

In [11]:
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
import sys
import os

class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["vectorstore", "websearch"] = Field(
        ...,
        description="Given a user question choose to route it to web search or a vectorstore.",
    )

llm = llm_model

structured_llm_router = llm.with_structured_output(RouteQuery)

system = """You are an expert at routing a user question to a vectorstore or web search.
The vectorstore contains documents related to covid, diabetes, anemaia.
Use the vectorstore for questions on these topics. For any other health related questions , use web-search. 
For all other questions, do not answer and give reason"""

route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

question_router = route_prompt | structured_llm_router

In [12]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )

llm = llm_model
structured_llm_grader = llm.with_structured_output(GradeDocuments)

system = """You are a grader assessing relevance of a retrieved document to a user question. \n 
    If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""

grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader

In [13]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableSequence
from pydantic import BaseModel, Field

class GradeAnswer(BaseModel):

    binary_score: bool = Field(
        description="Answer addresses the question, 'yes' or 'no'"
    )

llm =  llm_model
structured_llm_grader = llm.with_structured_output(GradeAnswer)

system = """You are a grader assessing whether an answer addresses / resolves a question \n 
     Give a binary score 'yes' or 'no'. Yes' means that the answer resolves the question."""
answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "User question: \n\n {question} \n\n LLM generation: {generation}"),
    ]
)

answer_grader: RunnableSequence = answer_prompt | structured_llm_grader

In [14]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser

llm = llm_model
prompt = hub.pull("rlm/rag-prompt")
generation_chain = prompt | llm | StrOutputParser()

In [15]:
def retrieve(state: GraphState) -> Dict[str, Any]:
    """Retrieve documents from vector store."""
    print("---RETRIEVE---")
    question = state["question"]
    documents = retriever.invoke(question)
    return {"documents": documents, "question": question}

def grade_documents(state: GraphState) -> Dict[str, Any]:
    """
    Determines whether the retrieved documents are relevant to the question
    If any document is not relevant, we will set a flag to run web search

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Filtered out irrelevant documents and updated web_search state
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    filtered_docs = []
    web_search = False
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score.binary_score
        if grade.lower() == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            web_search = True
            continue
    return {"documents": filtered_docs, "question": question, "web_search": web_search}

def generate(state: GraphState) -> Dict[str, Any]:
    """Generate answer using documents and question."""
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]
    generation = generation_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}

In [16]:
# Workflow node identifiers
RETRIEVE = "retrieve"
GRADE_DOCUMENTS = "grade_documents"  
GENERATE = "generate"
WEBSEARCH = "websearch"

In [19]:
from langgraph.graph import END, StateGraph

def decide_to_generate(state):
    """Route to web search or generation."""
    print("---ASSESS DOCUMENTS---")
    return WEBSEARCH if state["web_search"] else GENERATE

def route_question(state: GraphState) -> str:
    """Route question to vectorstore or web search."""
    print("---ROUTE QUESTION---")
    source: RouteQuery = question_router.invoke({"question": state["question"]})
    return WEBSEARCH if source.datasource == WEBSEARCH else RETRIEVE


# Build workflow
workflow = StateGraph(GraphState)
workflow.add_node(RETRIEVE, retrieve)
workflow.add_node(GRADE_DOCUMENTS, grade_documents)
workflow.add_node(GENERATE, generate)
workflow.add_node(WEBSEARCH, web_search)

workflow.set_conditional_entry_point(
    route_question,
    {WEBSEARCH: WEBSEARCH, RETRIEVE: RETRIEVE},
)

workflow.add_edge(RETRIEVE, GRADE_DOCUMENTS)
workflow.add_conditional_edges(
    GRADE_DOCUMENTS,
    decide_to_generate,
    {WEBSEARCH: WEBSEARCH, GENERATE: GENERATE},
)

workflow.add_edge(WEBSEARCH, GENERATE)

app = workflow.compile()



In [20]:
def format_response(result):
    """Extract response from workflow result."""
    if isinstance(result, dict) and "generation" in result:
        return result["generation"]
    elif isinstance(result, dict) and "answer" in result:
        return result["answer"]
    else:
        return str(result)


In [21]:
print("Adaptive RAG System")
print("Type 'quit' to exit.\n")

while True:
    try:
        question = input("Question: ").strip()
        
        if question.lower() in ['quit', 'exit', 'q', '']:
            break
            
        print("Processing...")
        result = None
        for output in app.stream({"question": question}):
            for key, value in output.items():
                result = value
                
        if result:
            print(f"\nAnswer: {format_response(result)}")
        else:
            print("No response generated.")
            
    except KeyboardInterrupt:
        break
    except Exception as e:
        print(f"Error: {str(e)}")

Adaptive RAG System
Type 'quit' to exit.



Question:  What is LLM?


Processing...
---ROUTE QUESTION---
---WEB SEARCH---
---GENERATE---

Answer: Large language models (LLMs) are a type of artificial intelligence (AI) program or machine learning model. They are designed to comprehend, process, understand, and generate human language text. LLMs are trained on immense amounts of data, enabling them to perform a wide range of language-related tasks.


Question:  My fasting sugar is less than 60mg/dL


Processing...
---ROUTE QUESTION---
---RETRIEVE---
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS DOCUMENTS---
---GENERATE---

Answer: I don't know the answer. The provided context discusses Type 1 and Type 2 diabetes, their symptoms, causes, diagnosis, and treatment, as well as general information about blood sugar levels. However, it does not specify what a fasting sugar level less than 60mg/dL indicates.


Question:  My blood sugar level is 200mg/dL


Processing...
---ROUTE QUESTION---
---RETRIEVE---
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS DOCUMENTS---
---GENERATE---

Answer: I don't know the answer based on the provided context. The documents discuss type 1 and type 2 diabetes and mention "unusual blood sugar levels" but do not specify what a 200mg/dL level indicates.


Question:  symptoms of type 1 diabetes


Processing...
---ROUTE QUESTION---
---RETRIEVE---
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS DOCUMENTS---
---WEB SEARCH---
---GENERATE---

Answer: Symptoms of type 1 diabetes can appear quickly and include extreme thirst, increased hunger, and frequent urination. Other common signs are unexplained weight loss, feeling tired or weak, dry mouth, and mood changes. Some individuals may also experience upset stomach, vomiting, vision changes, or repeated infections.


Question:  quit
